# ***GPT FROM SCRATCH: CHAR LEVEL GPT-2***

### Libs

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
from gpt.bigram import BigramModel

### Input file: all Shakespeare books

In [11]:
text = open("input.txt", 'r', encoding='utf-8').read()
len(text)

1115394

### Text sample

In [12]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Vocabulary and encode/decode

In [13]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"{vocab_size=}\n{chars=} \n")

#Create a dictionary mapping each character to a unique integer
stoi = {ch: i for i,ch in enumerate(chars)}
itos = {v:k for k,v in stoi.items()}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode("encode this text"))
print(decode(encode("encode this text")))

vocab_size=65
chars=['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 

[43, 52, 41, 53, 42, 43, 1, 58, 46, 47, 57, 1, 58, 43, 62, 58]
encode this text


### Train Test Val Split

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
def train_test_val_split(data: str,
                         train_size: float = 0.8,
                         val_size: float = 0.1,
                         test_size: float = 0.1):
    assert train_size + val_size + test_size == 1
    data_len = len(data)
    train_len = int(data_len * train_size)
    val_len = int(data_len * val_size)
    test_len = data_len - train_len - val_len
    train_data = data[:train_len]
    val_data = data[train_len:train_len+val_len]
    test_data = data[train_len+val_len:]
    return train_data, val_data, test_data

train_data, val_data, test_data = train_test_val_split(data, 0.8, 0.2, 0)

In [7]:
block_size = 8
torch.manual_seed(42)
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


### Take batches of len of block_size

In [8]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(f"{xb=}")
print(f"{yb=}")

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

xb=tensor([[47, 57, 10,  1, 39, 52, 42,  1],
        [59, 56,  1, 46, 43, 39, 56, 58],
        [32, 46, 39, 58,  1, 39, 50, 61],
        [26, 53, 58, 46, 47, 52, 45,  1]])
yb=tensor([[57, 10,  1, 39, 52, 42,  1, 50],
        [56,  1, 46, 43, 39, 56, 58, 57],
        [46, 39, 58,  1, 39, 50, 61, 39],
        [53, 58, 46, 47, 52, 45,  1, 40]])
when input is [47] the target: 57
when input is [47, 57] the target: 10
when input is [47, 57, 10] the target: 1
when input is [47, 57, 10, 1] the target: 39
when input is [47, 57, 10, 1, 39] the target: 52
when input is [47, 57, 10, 1, 39, 52] the target: 42
when input is [47, 57, 10, 1, 39, 52, 42] the target: 1
when input is [47, 57, 10, 1, 39, 52, 42, 1] the target: 50
when input is [59] the target: 56
when input is [59, 56] the target: 1
when input is [59, 56, 1] the target: 46
when input is [59, 56, 1, 46] the target: 43
when input is [59, 56, 1, 46, 43] the target: 39
when input is [59, 56, 1, 46, 43, 39] the target: 56
when input is [59, 56

### Bigram char-level model

In [9]:
class BigramModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # Embedding table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx: (batch_size, block_size)
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # Channel must b 2nd dim
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # Only last prediction
            probs = F.softmax(logits, dim=-1)
            # Takes one new token and add it to the idx
            idx = torch.cat([idx, torch.multinomial(probs, num_samples=1)], dim=1)
        return idx

In [10]:
m = BigramModel(vocab_size)
logits, loss = m(xb, yb)
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long),
                        max_new_tokens = 100)[0].tolist()))


F$z
E?kFwu
'buM-,Y3fYNsA3xp.mpsAqaZ-RUldc;F
M$GfYCUCkFO-bJbz-R;O!slp.FNsJDV'jRzIMQ'EdRbqAoWTjrniaIIa


### Optimizing it

In [11]:
optim = torch.optim.AdamW(m.parameters(), lr=0.001)

batch_size = 32
n_steps = 30000
for _ in range(n_steps):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    loss.backward()
    optim.step()
    optim.zero_grad()
    if _ % (n_steps//10) == 0:
        print(f"Loss: {loss.item()}")
        print(decode(m.generate(idx = xb[:1], max_new_tokens = 100)[0].tolist()))

Loss: 4.702682971954346
, how I U?OlqbAoCjULUlwyVfGdsujrfc-j&Br,VkvPT ;mVDFNvPNvH-EsAa$dJTVwrx fK-d3fluagUuTVlsN.d?VLUlvIeNsxqaIOlVE
Loss: 2.7912943363189697
t wrapt bOhyVYotoun:JbJy, budutUSbr!OMJ, O-
Pr

O:liM3bkLADUy.OI wedeTOR?
HA;


CKaSHARGENIZcy onoA
Tourel-k
Loss: 2.5110576152801514
ur highnchJ! hour, w wIIfr!
Wharersw s ayo yo d ;


wet charilkse thtwkimburang bendetharsRf t reix ncWhevev
Loss: 2.283414840698242
t,
And hon tr out irerond; ben, squa cinol,
f! wind me ar he pliear, RDUTh t, My ffourere y. w
S:
An
CKerd y
Loss: 2.3937628269195557
as,
Thanodore, METhitHEEShoue ard orelliver uran, hillvr, shied t?

Anjergngery-ugl alve ithad thyoupesspan.
Loss: 2.504157781600952
oung affil CI'e d arnonth'tegin as s y ar blveandave, p my ghasouririn nincrind frds staicand t blis d, outa
Loss: 2.4056339263916016
ard, not jeow meeeinge me.
Shatourgh s foor more? k?
Whe sed.
Th t?
OHarevertaseviod d, be free telouk-ingm 
Loss: 2.3617005348205566
I hope hmefimes.
D:



HAGERIf fy Lo bts

## Self-attention Mecanism

In [18]:
torch.manual_seed(42)
B,T,C = 4,8,2 # batch, time, channel
trill = torch.tril(torch.ones(T, T)) # Low Triangular matrix (decode)
x = torch.randn(B,T,C)

# Attention Head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
# Key, Query and Value
k = key(x)
q = query(x)
v = value(x)

# Attention
wei = q @ k.transpose(-2, -1) / head_size**0.5
print(f"{wei.shape=}")
wei = wei.masked_fill(trill == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
# wei = trill @ wei
# wei = wei.mean(dim=-1, keepdim=True)
print(wei.shape)
output = wei @ v


wei.shape=torch.Size([4, 8, 8])
torch.Size([4, 8, 8])


In [14]:
output.shape

torch.Size([4, 8, 16])